In [ ]:
from config import *
from twelvedata import TDClient
import pandas as pd
import time, concurrent.futures

td = TDClient(api_key)

In [ ]:
stocks = ["AAPL", "GOOGL", "AMZN", "PEP", "AMD"]

In [ ]:
def indicators(symbol):
    ts = td.time_series(symbol=symbol, interval='1min', outputsize=3)
    ind = ts.with_sma(time_period=5).with_sma(time_period=8).with_sma(time_period=13).as_pandas()
    ind['symbol'] = symbol
    return ind

In [ ]:
t0 = time.perf_counter()

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(indicators, symbol) for stock in stocks]
    
for result in results:
    print(result)

In [ ]:
print('total exec time = {}'.format(time.perf_counter() - t0))

In [ ]:
import alpaca_trade_api as alpaca
import requests, datetime

In [ ]:
t0 = datetime.datetime.now()

In [ ]:
r = requests.get(alpaca_base_url + '/v2/assets', headers=alpaca_headers)
assets = pd.DataFrame(r.json())
active = assets[assets.status.eq('active')]
activeTradable = active[active.tradable.eq(True)]
activeTradableUS = activeTradable.loc[activeTradable['class']=='us_equity']
activeTradableUS = activeTradableUS.loc[activeTradableUS['exchange']=='NASDAQ']
alp = alpaca.REST(alpaca_API_KEY, alpaca_SECRET_KEY, api_version='v2')

lastDayBars = pd.DataFrame()
while not activeTradableUS.empty:
    symbols = activeTradableUS['symbol'][:200]
    activeTradableUS = activeTradableUS.iloc[200:]
    barset = alp.get_barset(symbols, timeframe='day', limit=1, start=None, end=None, after=None, until=None)
    barsetSerie = pd.Series(barset)

    for index, value in barsetSerie.iteritems():
        lastDayBars =  lastDayBars.append(pd.DataFrame(data=[[value[0].t, index, value[0].o, value[0].c, value[0].h, value[0].l, value[0].v]],
                                                                   columns=['time', 'symbol','open', 'close', 'high', 'low', 'volume']))

lastDayBars = lastDayBars.sort_values(by=['volume'], ascending=False)
lastDayBars.to_csv('lastDayBars_{}.csv'.format(datetime.datetime.now().strftime("%Y-%m-%d")))

In [ ]:
print('total execution time = {}'.format(datetime.datetime.now() - t0))

In [ ]:
import json
import threading
from sqlalchemy import create_engine

class smaStrategy:
    def __init__(self, symbols=['INO', 'BIOC', 'AMD', 'QQQ', 'SQQQ', 'HTBX', 'AAPL', 'TRNX', 'MSFT', 'CZR']):
        self.alpaca = tradeapi.REST(alpaca_API_KEY, alpaca_SECRET_KEY, alpaca_base_url, 'v2')
        self.td = TDClient(apikey=twelvedata_api_key)
        self.symbols = symbols
        self.blacklist = set()
        self.outputsize = 2  # number of previous bars // see twelvedata indicators
        self.qty = 10  # number of shares to be executed evry transaction /// will have to adjust this
        self.sqlconn = None

    def run(self):
        # First, cancel any existing orders so they don't impact our buying power.
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
            self.alpaca.cancel_order(order.id)

        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")

        self.sqlconn = create_engine('sqlite:///smaDataFrame{}.db'.format(time.strftime("%d-%b-%Y")), echo=False)
        # Fetch the indicators every minute, making necessary trades ///only rsi and gain>0.1% for now.
        while self.alpaca.get_clock().is_open:
            df = self.batchTi()
            print('============================== New Cycle ============================================')
            print('current time = {}'.format(datetime.datetime.now()))
            print('-------------------------------------------------------------------------------------')
            print(df)
            print('-------------------------------------------------------------------------------------')
            #split stocks to sell and buy groups so we can send butch sell/buy --> alpaca paper trading
            ss = df[['symbol', 'signal']].drop_duplicates(keep='first')
            shortSymbols = ss[ss['signal'] == 'sell']['symbol'].tolist()
            longSymbols = ss[ss['signal'] == 'buy']['symbol'].tolist()
            respSell = []
            respBuy = []
            self.sendBatchOrder(self.qty, shortSymbols, 'sell', respSell)
            self.sendBatchOrder(self.qty, longSymbols, 'buy', respBuy)
            print('-------------- Cycle ended, Next update in 60 seconds --------------------------------')
            df.to_sql('Candle Sticks', con=self.sqlconn, if_exists='append')
            print('\n')
            print('\n')
            print('\n')
            time.sleep(60)

        print('Market closed for today')

    # Wait for market to open.
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while (not isOpen):
            clock = self.alpaca.get_clock()
            openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
            currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
            timeToOpen = int((openingTime - currTime) / 60)
            print(str(timeToOpen) + " minutes til market open.")
            time.sleep(60)
            isOpen = self.alpaca.get_clock().is_open

    # time series indicators for symbols : list of symbols [sym1, sm2, ...., symN]
    def batchTi(self):

        with concurrent.futures.ThreadPoolExecutor() as ex:
            tis = [ex.submit(self.timeInd, symbolId) for symbolId in range(len(self.symbols))]

        df = pd.DataFrame()
        for f in concurrent.futures.as_completed(tis):
            df = df.append(f.result())

        return df

    # return time_series indicator (ti) of a specific symbol stock
    def timeInd(self, symbolID=0):
        ts = self.td.time_series(symbol=self.symbols[symbolID], interval='1min', outputsize=3)
        ti = ts.with_sma(time_period=5).with_sma(time_period=8).with_sma(time_period=13).as_pandas()
        ti['symbol'] = self.symbols[symbolID]
        #trying to make up the trade signal buy/sell/None
        if not ti.empty and ti['sma1'][0] > ti['sma2'][0] \
                and ti['sma1'][-1] < ti['sma2'][-1] \
                and ti['sma1'][0] > ti['sma3'][0]:
            ti['signal'] = 'buy'
        elif not ti.empty and ti['sma1'][0] < ti['sma2'][0] \
                and ti['sma1'][-1] > ti['sma2'][-1] \
                and ti['sma2'][0] > ti['sma3'][0]:
            ti['signal'] = 'sell'
        else:
            ti['signal'] = None

        return ti

    # Submit a batch order that returns completed and uncompleted orders.
    def sendBatchOrder(self, qty, stocks, side, resp):
        executed = []
        incomplete = []
        for stock in stocks:
            if (self.blacklist.isdisjoint({stock})):
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder, args=[qty, stock, side, respSO])
                tSubmitOrder.start()
                tSubmitOrder.join()
                # if(not respSO[0]):
                #    # Stock order did not go through, add it to incomplete.
                #    incomplete.append(stock)
                # else:
                #    executed.append(stock)
                # respSO.clear()
            resp.append([executed, incomplete])

    # Submit an order if buy or sell conditions are met.
    def submitOrder(self, qty, stock, side, resp):
        position = requests.get(alpaca_base_url + '/v2/positions/' + stock, headers=alpaca_headers)
        # Submit buy order only if ownes less than 5*qty shares in the portfolio
        if ('buy' in side and (position.status_code == 404 or float(position.json()['qty']) < (self.qty * 5))):
            try:
                self.alpaca.submit_order(stock, qty, side, "market", "day")
                print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
                resp.append(True)
            except:
                print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
                resp.append(False)
        # Sell only if we own the asset and the profit is higher than 0.1%
        elif ('sell' in side and (
                position.status_code == 200 and float(position.json()['unrealized_plpc']) > (0.1 / 100))):
            try:
                self.alpaca.submit_order(stock, qty, side, "market", "day")
                print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
                resp.append(True)
            except:
                print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
                resp.append(False)
        else:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
            resp.append(True)


t0 = time.perf_counter()

sma = smaStrategy()
sma.run()

print('total exec time = {}'.format(time.perf_counter() - t0))